In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [ ]:
import expectexception

# Exceptions

Exceptions are Python's mechanism for interrupting the normal flow of the program.  In standard operation, contexts are pushed onto the stack at function calls and popped off when the function returns.  Exceptions halt the execution of the program and then bubble up until they are handled.  If they reach the interpreter's context, execution halts and the interpreter prints the exception and a traceback.

Exeptions are generally used for "unexpected" occurances, but the Python community is generally pretty liberal in defining what is meant by that.  Anything that's somewhat unusual is generally considered fair game for an exception.

For example, dividing by zero is not allowed.  Some languages might return a NaN, but Python raises an exception.

In [ ]:
%%expect_exception ZeroDivisionError

1 / 0

Exceptions are handled with a try/except block.

In [ ]:
try:
    1 / 0
except ZeroDivisionError:
    print "Can't do that!"

`ZeroDivisionError` is a class, like all exceptions.  The except block will only run if the exception is of that class.

In [ ]:
%%expect_exception ZeroDivisionError

try:
    1 / 0
except RuntimeError:
    print "Can't do that!"

You can use multiple except blocks to catch and handle different types of exceptions.

In [ ]:
try:
    1 / 0
except RuntimeError:
    print "Something went wrong"
except ZeroDivisionError:
    print "Can't do that!"

Or you can list multiple exception classes in a tuple, and the except block will catch an exception of any of those types.

In [ ]:
try:
    1 / 0
except (RuntimeError, ZeroDivisionError):
    print "Something went wrong!"

## Catching general exceptions

It is usually good practice to catch only those exceptions that you expect.  Generally, the except block will attempt to fix whatever caused the exception, allowing execution to continue.  If you catch an error you don't expect, you probably won't be able to recover correctly, and you will leave the program in an unanticipated state.

Nonetheless, there are some cases where catching all exceptions is a good idea.  Generally, this happens at a high level in a program that needs to keep running.  In these cases, the exception handler doesn't attempt to fix the problem -- it will just reset things to a known good state and then continue.  For example, a web server may catch all exceptions raised by a handler and return a 500 code, rather than dying in response to an unexpected exception.

A bare except statement will catch everything.  You should **never do this!**

In [ ]:
try:
    1 / 0
except:  # NO NO NO NO NO
    print "Something went wrong!"

The reason for this prohibition is that Python uses the exception handling framework to manage some other functionality.  For example, the `sys.exit()` function, which should cause a program to exit, raises `SystemExit`.  If this happens in a bare except block, you won't actually exit.

In [ ]:
import sys
try:
    sys.exit(1)
except:
    print "Something went wrong!"

Instead, take advantage of the fact that all true exceptions inherit from the `Exception` class.

In [ ]:
print issubclass(ZeroDivisionError, Exception)
print issubclass(SystemExit, Exception)

In [ ]:
try:
    1 / 0
except Exception:
    print "Something went wrong!"

## Handling success

For similar reasons, only code that you expect to raise exceptions should live within the try block.  Otherwise the error you think you are reacting to may not actually be the error that occured.

Often times, you have code that should only run if a previous line exected without an error.  Instead of putting all of this code in the try block, put it in an else clause.  This block will run if and only if the try block did not raise an Exception.

In [ ]:
try:
    0 / 1
except ZeroDivisionError:
    print "Oh no!"
else:
    print "Oh yes!"

Sometimes you want some code to run regardless of whether an exception has been raised.  For example, you may want to close a file handle before exiting.  The finally block will always run, even if there was an unhandled exception.

In [ ]:
%%expect_exception ZeroDivisionError

try:
    1 / 0
finally:
    print "Runs regardless"

All of these various clauses can be combined into a single try block.

In [ ]:
try:
    1 / 0
except RuntimeError:
    print "Something went wrong"
except ZeroDivisionError:
    print "Oh no!"
else:
    print "Oh yes!"
finally:
    print "Runs regardless"

## Doing something with the error

Sometimes you need to know more than just what type the error was.  Use the `as` keyword to assign the exception instance to a variable for further inspection.

In [ ]:
try:
    1 / 0
except ZeroDivisionError as error:
    print error

Sometimes you may want to pass an error upwards in the call stack.  You can re-raise an exception with the `raise` keyword.

In [ ]:
%%expect_exception ZeroDivisionError

try:
    1 / 0
except ZeroDivisionError:
    print "Bad things happened"
    raise

## Raising errors

You can also raise your own exceptions with the `raise` keyword.

In [ ]:
%%expect_exception RuntimeError

raise RuntimeError

Exceptions take (at least) an argument descripting what went wrong.

In [ ]:
%%expect_exception RuntimeError

raise RuntimeError("I had some trouble")

You can define custom exception types by subclassing `Exception` or one of its subclasses.  Ususally, there's nothing to define in the class.

In [ ]:
class CustomError(Exception):
    pass

In [ ]:
%%expect_exception CustomError

raise CustomError("Something special")

## Exceptions and the call stack

Exceptions truly shine in their interaction with the **call stack**.  Like many other languages, Python keeps a [stack](https://en.wikipedia.org/wiki/Stack_%28abstract_data_type%29) of function contexts during execution.  Each time a function is called, a new context for that function is pushed onto the stack.  When that function returns, the corresponding context is popped off the stack.

For a concrete example, consider the following code:

In [ ]:
def sqrt(z):
    return z ** 0.5

def twice_sqrt(y):
    val = sqrt(y)
    return 2 * val

def twice_sqrt_plus_one(x):
    val = twice_sqrt(x)
    return val + 1

twice_sqrt_plus_one(9)

Let's examine what happens when `twice_sqrt_plus_one(9)` is called.

1. A context for twice_sqrt_plus_one is pushed to the stack, with x=9.
2. `twice_sqrt(9)` is called.
3. A context for twice_sqrt is pushed to the stack, with y=9.
4. `sqrt(9)` is called.
5. A context for sqrt is pushed to the stack, with z=9.
6. The sqrt function returns 3.0.
7. The context for sqrt is popped from the stack, and in the context for twice_sqrt, val=3.0.
8. The twice_sqrt fuction returns 6.0.
9. The context for twice_sqrt is popped from the stack, and in the context for twice_sqrt_plus_one, val=6.0.
10. The twice_sqrt_plus_one function returns 7.0.
11. The context for twice_sqrt_plus_one is popped from the stack.  This returns execution to the Jupyter notebook, which happens to print the returned value.

You can step through this process at [pythontutor.com](http://www.pythontutor.com/visualize.html#code=def%20sqrt(z%29%3A%0A%20%20%20%20return%20z%20**%200.5%0A%0Adef%20twice_sqrt(y%29%3A%0A%20%20%20%20val%20%3D%20sqrt(y%29%0A%20%20%20%20return%202%20*%20val%0A%0Adef%20twice_sqrt_plus_one(x%29%3A%0A%20%20%20%20val%20%3D%20twice_sqrt(x%29%0A%20%20%20%20return%20val%20%2B%201%0A%0Atwice_sqrt_plus_one(9%29&cumulative=false&curInstr=0&heapPrimitives=false&mode=display&origin=opt-frontend.js&py=2&rawInputLstJSON=%5B%5D&textReferences=false).

(Incidentally, this is how Python can handle recursion without getting confused.  Each recursive call of the function gets its own context on the stack with its own set of local variables.)

What happens if we try to take the square root of a negative number?

In [ ]:
%%expect_exception ValueError

(-9) ** 0.5

If an exception is not handled at the top level of the call stack, that context is popped and level below is given a chance to handle the exception.  If that level can't handle it, then it is popped, and....

In [ ]:
%%expect_exception ValueError

twice_sqrt_plus_one(-9)

This exception has bubbled all the way up to the intepreter context, so a traceback is printed.  This traceback contains a record of all the contexts that could not handle the exception.  The last entry shows where the exception was actually raised, and the previous entries show where the function calls that lead to this exception were made.

We could have caught this exception at any level in the call stack.  For example:

In [ ]:
def twice_sqrt_plus_one_safe(x):
    try:
        val = twice_sqrt(x)
    except ValueError:
        val = 0
    return val + 1

twice_sqrt_plus_one_safe(-9)

That try/except block catches the value error thrown by the power expression inside the sqrt function inside the twice_sqrt function and recovers to give us a value.  You can [visualize this execution](http://www.pythontutor.com/visualize.html#code=def%20sqrt(z%29%3A%0A%20%20%20%20return%20z%20**%200.5%0A%0Adef%20twice_sqrt(y%29%3A%0A%20%20%20%20val%20%3D%20sqrt(y%29%0A%20%20%20%20return%202%20*%20val%0A%0Adef%20twice_sqrt_plus_one_safe(x%29%3A%0A%20%20%20%20try%3A%0A%20%20%20%20%20%20%20%20val%20%3D%20twice_sqrt(x%29%0A%20%20%20%20except%20ValueError%3A%0A%20%20%20%20%20%20%20%20val%20%3D%200%0A%20%20%20%20return%20val%20%2B%201%0A%0Atwice_sqrt_plus_one(-9%29&cumulative=false&curInstr=0&heapPrimitives=false&mode=display&origin=opt-frontend.js&py=2&rawInputLstJSON=%5B%5D&textReferences=false) too.

The behavior allows exceptions to be dealt with at the proper level.  Low level code doesn't have to worry about recovering in the "right way"&mdash;it can just throw an exception and let the high level code figure out what needs to be done.

Some languages use an alternative approach where function should return a flag indicating that an error occured.  This requires each function to check this flag for each function that it calls and set its own flag appropriately.  If just one function in a stack forgets to do this, errors can get lost.  Python's exception system doesn't require such handling and is more robust.

The ethos of Python is to take advantage of this ability&mdash;exceptions are not something that need to be avoided at all costs.  It is generally considered better to try something and catch the resulting exception than to set up a bunch of if statements to see if something will succeed.  For example, instead of testing whether a key is in a dictionary before using it, you can just try to index with it and catch the `KeyError` if it isn't.  (Although it's better still to use the dictionary's `.get()` method.)  After all, *it's easier to ask for forgiveness than permission.*

## Reading tracebacks

Sooner or later, you will make a mistake and Python will spit out a huge traceback.  Now you have to try to track down what went wrong in this call stack.  It may look hopeless, but don't panic!  The information you need is somewhere in the traceback.  The key is to figure out what to ignore.

For an example, consider this (rather poorly-written) code:

In [ ]:
import pandas as pd

def make_series(n):
    s = str(n)
    partials = [s[:i+1] for i in xrange(len(s))]
    series = pd.Series(partials, dtype=int)
    return series.values

def make_dataframe(l):
    df = pd.DataFrame({'n': l})
    df['partials'] = df.applymap(make_series)['n']
    return df

The `make_dataframe` takes a list of integers and will make make a DataFrame with two columns: **n** has the integers and **partials** a list of integers made by taking the first digit, the first two digits, etc., of **n**.

In [ ]:
make_dataframe([12, 927, 5])

Seems to work.  Let's try another input.

In [ ]:
%%expect_exception ValueError

make_dataframe([18, -2005, 35])

Oh, my.  The error says something about `long()` (huh?) and base 10 (huh$^2$?), and it seems to have occured in the creation of a numpy array (huh$^3$?).  Why are any of these things happening?  We didn't explictly refer to any of them.

Many of the levels of the call stack are within the Pandas library.  At the start, we can probably assume that Pandas is working properly and we did something wrong.  So let's ignore all levels of the stack that are from Pandas and figure out what we did wrong.  A good place to start is to find the deepest code in the traceback that we are responsible for.  In this case, it happens to be line 6 inside of `make_series()`:

    ----> 6     series = pd.Series(partials, dtype=int)

Already, this tells us useful details.  Evidently, we were able to construct the DataFrame and call `.applymap()` on it.  The problem is in that function that we put into `.applymap()`.  And specifically, it's making that Series that is causing problems.  Let's figure out which input is giving us grief.

In [ ]:
for n in [18, -2005, 35]:
    try:
        make_series(n)
    except ValueError:
        print "%i is the problem!" % n

Okay.  Let's figure out what happens inside `make_series` in this case.

In [ ]:
s = str(-2005)
[s[:i+1] for i in xrange(len(s))]

See the problem now?  The string `'-'` isn't a valid integer.  This is causing the exception.  I guess we need to do something better to handle negative numbers.

### Using the debugger

Debuggers offer a way to look through the stack when something's gone wrong, instead of conducting a *post mortem* analysis.  The context of each level of the call stack is available for examination live.

Usually you have to launch your code specifically in the debugger to get this capability.  (For Python, the command is `pdb`.)  But, because Jupyter is awesome, it offers you the ability to examine exceptions that occur in the notebook.

To launch the debugger on the previous traceback, uncomment and run this cell. Can you use the debugger to dianose the problem?

In [ ]:
#debug

Some hints:
- Use *u* and *d* to move up and down the call stack
- Use *p &lt;name&gt;* to print out the variable with &lt;name&gt;
- Use *?* to see additional commands.
- The debugger is actually a full python shell, so you can run Python expressions.
- Use *q* to quit the debugger session.

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*